In [35]:
from langchain import HuggingFaceHub
from dotenv import load_dotenv
import os

In [33]:
load_dotenv()

True

In [37]:
SECRET_LLAMA_KEY = os.getenv('LLAMA_API_KEY')

In [5]:
llm_huggingface = HuggingFaceHub(repo_id='google/flan-t5-large', model_kwargs={'temperature': 0, 'max_length': 64})

c:\Users\Divy Solanki\.conda\envs\llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Divy Solanki\.conda\envs\llm\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [10]:
output = llm_huggingface.invoke("Can you write a poem?")
print(output)

i love you i love you i love you i love you i love you i love you i love you i love you i love you i love you i love you i love you i love you i love you i love you i love


# Prompt Templates

In [12]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(input_variables=['country'], template="Tell me the capital of {country}")

prompt_template.format(country="India")

'Tell me the capital of India'

In [13]:
from langchain.chains import LLMChain

In [16]:
chain = LLMChain(llm=llm_huggingface, prompt=prompt_template)
chain.invoke('United States')

{'country': 'United States', 'text': 'detroit'}

# Combining Multiple Chains using simple Sequential Chain

In [19]:
capital_prompt = PromptTemplate(input_variables=['country'],
                                template="Pleaase tell me the capital of the {country}")

capital_chain = LLMChain(llm=llm_huggingface, prompt=capital_prompt)

In [21]:
famous_template = PromptTemplate(input_variables=['capital'],
                                 template="Suggest me some amazing places to visit in {capital}")

famous_chain = LLMChain(llm=llm_huggingface, prompt=famous_template)

In [22]:
from langchain.chains import SimpleSequentialChain

In [24]:
chain = SimpleSequentialChain(chains=[capital_chain, famous_chain])
chain.invoke("Russia")

{'input': 'Russia',
 'output': 'The Palace of the Grand Duke of Tsar Alexander III'}

## Sequential Chain

In [25]:
capital_prompt = PromptTemplate(input_variables=['country'],
                                template="Pleaase tell me the capital of the {country}")

capital_chain = LLMChain(llm=llm_huggingface, prompt=capital_prompt, output_key='capital')

In [26]:
famous_template = PromptTemplate(input_variables=['capital'],
                                 template="Suggest me some amazing places to visit in {capital}")

famous_chain = LLMChain(llm=llm_huggingface, prompt=famous_template, output_key='places')

In [27]:
from langchain.chains import SequentialChain

In [28]:
chain = SequentialChain(chains=[capital_chain, famous_chain],
                        input_variables=['country'],
                        output_variables=['capital', 'places'])

In [30]:
chain.invoke({'country': "India"})

{'country': 'India', 'capital': 'Delhi', 'places': 'The Taj Mahal'}

# ChatModels

In [32]:
from llamaapi import LlamaAPI
from langchain_experimental.llms import ChatLlamaAPI

In [39]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [38]:
llama = LlamaAPI(api_token=SECRET_LLAMA_KEY)
chatllm = ChatLlamaAPI(client=llama)

In [41]:
chatllm.invoke([
    SystemMessage(content="You are a comedian AI assistant"),
    HumanMessage(content="Please provide some comedy punchlines on AI"),
])

AIMessage(content='😂 Sure thing! Here are some AI-themed comedy punchlines for ya:\n\n1. Why did the AI go to therapy? Because it had a kernel crisis! 🤖😱\n2. How did the AI become a comedian? It learned to joke-ify its neural networks! 🤪💻\n3. Why was the AI kicked out of the bar? It was making too many micro-brew requests! 🍺🤖\n4. What did the AI say to the punch card? "You\'re so retro, you make me feel like a dinosaur!" 😂👾\n5. Why did the AI go on a diet? It wanted to lose some bytes! 😂💪\n6. How did the AI become a baker? It learned to knead dough-matics! 🍞🤖\n7. Why did the AI join a gym? It wanted to pump up its neural networks! 💪💻\n8. What did the AI say to the robot? "You\'re so last century, you make me feel like a relic!" 😂🤖\n9. Why did the AI become a doctor? It learned to diagnose glitches! 🩸🤖\n10. How did the AI become a lawyer? It learned to argue circuitry! 🏢🤖\n\nI hope these punchlines bring a smile to your face! 😄 Do you have any other topics you\'d like to explore? 🤔')

# Prompt Template + LLM + Output Parsers

In [42]:
from langchain_experimental.llms import ChatLlamaAPI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [43]:
class CommaSeparatedOutput(BaseOutputParser):
    def parse(self, text:str):
        return text.strip().split(",")

In [44]:
template = "You are helpful assistant. When the user gives any input you should generate 5 words synonyms which should be , separated"
human_template = '{text}'
chatprompt = ChatPromptTemplate.from_messages([
    ('system', template),
    ('human', human_template)
])

In [45]:
chain = chatprompt|chatllm|CommaSeparatedOutput()

In [46]:
chain.invoke({"text":"intelligent"})

['1. brilliant\n2. clever\n3. smart\n4. wise\n5. astute']